# "Kick" Dataset

In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency


In [2]:
from sklearn.datasets import fetch_openml

data = fetch_openml(data_id=41162, parser='auto')

# The returned dataset is a Bunch object, similar to a dictionary
X = data['data']
y = data['target']

In [3]:
# Summary vectors creation

default_summary  = []
encoder_summary  = []
early_stopping_summary = []
value_summary    = []
time_summary     = []
n_models_summary = []
card_9_summary   = []

### Description

One of the biggest challenges of an auto dealership purchasing a used car at an auto auction is the risk of that the vehicle might have serious issues that prevent it from being sold to customers. The auto community calls these unfortunate purchases "kicks".

Kicked cars often result when there are tampered odometers, mechanical issues the dealer is not able to address, issues with getting the vehicle title from the seller, or some other unforeseen problem. Kick cars can be very costly to dealers after transportation cost, throw-away repair work, and market losses in reselling the vehicle.

Modelers who can figure out which cars have a higher risk of being kick can provide real value to dealerships trying to provide the best inventory selection possible to their customers.

The challenge of this competition is to predict if the car purchased at the Auction is a Kick (bad buy).

In [4]:
X.head()

,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1260144000,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,AUTO,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,1260144000,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,AUTO,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,1260144000,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,MAROON,AUTO,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,1260144000,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,SILVER,AUTO,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,1260144000,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,SILVER,MANUAL,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


In [5]:
X.columns

Index(['PurchDate', 'Auction', 'VehYear', 'VehicleAge', 'Make', 'Model',
       'Trim', 'SubModel', 'Color', 'Transmission', 'WheelTypeID', 'WheelType',
       'VehOdo', 'Nationality', 'Size', 'TopThreeAmericanName',
       'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
       'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
       'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
       'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice',
       'PRIMEUNIT', 'AUCGUART', 'BYRNO', 'VNZIP1', 'VNST', 'VehBCost',
       'IsOnlineSale', 'WarrantyCost'],
      dtype='object')

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   PurchDate                          72983 non-null  int64   
 1   Auction                            72983 non-null  category
 2   VehYear                            72983 non-null  int64   
 3   VehicleAge                         72983 non-null  int64   
 4   Make                               72983 non-null  category
 5   Model                              72983 non-null  category
 6   Trim                               70623 non-null  category
 7   SubModel                           72975 non-null  category
 8   Color                              72975 non-null  category
 9   Transmission                       72974 non-null  category
 10  WheelTypeID                        69814 non-null  category
 11  WheelType                          69809 

In [7]:
X.shape

(72983, 32)

In [8]:
X.describe()

,PurchDate,VehYear,VehicleAge,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,VehBCost,WarrantyCost
count,7.298300e+04,72983.000000,72983.000000,72983.000000,72965.000000,72965.000000,72965.000000,72965.000000,72668.000000,72668.000000,72668.000000,72668.000000,72915.000000,72983.000000
mean,1.263092e+09,2005.343052,4.176644,71499.995917,6128.909217,7373.636031,8497.034332,9850.928240,6132.081287,7390.681827,8775.723331,10145.385314,6729.249949,1276.580985
std,1.819602e+07,1.731252,1.712210,14578.913128,2461.992768,2722.491986,3156.285284,3385.789541,2434.567723,2686.248852,3090.702941,3310.254351,1764.962643,598.846788
min,1.231114e+09,2001.000000,0.000000,4825.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,462.000000
25%,1.248048e+09,2004.000000,3.000000,61837.000000,4273.000000,5406.000000,6280.000000,7493.000000,4275.000000,5414.000000,6536.000000,7784.000000,5430.000000,837.000000
50%,1.264032e+09,2005.000000,4.000000,73361.000000,6097.000000,7303.000000,8444.000000,9789.000000,6062.000000,7313.000000,8729.000000,10103.000000,6700.000000,1155.000000
75%,1.279066e+09,2007.000000,5.000000,82436.000000,7765.000000,9021.000000,10651.000000,12088.000000,7736.000000,9013.000000,10911.000000,12309.000000,7900.000000,1623.000000
max,1.293667e+09,2010.000000,9.000000,115717.000000,35722.000000,36859.000000,39080.000000,41482.000000,35722.000000,36859.000000,39080.000000,41062.000000,45469.000000,7498.000000


Check the unique values of all variables.

In [9]:
X.nunique().sort_values(ascending=False)

VehOdo                               39947
MMRAcquisitonRetailCleanPrice        13456
MMRCurrentRetailCleanPrice           13192
MMRAcquisitionRetailAveragePrice     12725
MMRCurrentRetailAveragePrice         12493
MMRAcquisitionAuctionCleanPrice      11379
MMRCurrentAuctionCleanPrice          11265
MMRAcquisitionAuctionAveragePrice    10342
MMRCurrentAuctionAveragePrice        10315
VehBCost                              2010
Model                                 1063
SubModel                               863
PurchDate                              517
WarrantyCost                           281
VNZIP1                                 153
Trim                                   134
BYRNO                                   74
VNST                                    37
Make                                    33
Color                                   16
Size                                    12
VehicleAge                              10
VehYear                                 10
WheelTypeID

We check for duplicate rows.

In [10]:
X.duplicated().sum()

0

## Study of NA's

In [11]:
X.isna().sum().sort_values(ascending = False)

AUCGUART                             69564
PRIMEUNIT                            69564
WheelType                             3174
WheelTypeID                           3169
Trim                                  2360
MMRCurrentRetailCleanPrice             315
MMRCurrentRetailAveragePrice           315
MMRCurrentAuctionCleanPrice            315
MMRCurrentAuctionAveragePrice          315
VehBCost                                68
MMRAcquisitonRetailCleanPrice           18
MMRAcquisitionRetailAveragePrice        18
MMRAcquisitionAuctionCleanPrice         18
MMRAcquisitionAuctionAveragePrice       18
Transmission                             9
Color                                    8
SubModel                                 8
Size                                     5
TopThreeAmericanName                     5
Nationality                              5
VNZIP1                                   0
VNST                                     0
IsOnlineSale                             0
BYRNO      

As can be seen, there are 2 variables whose 95% are np.nan. Therefore we will eliminate these two features. The rest, we will impute them in our preprocessing pipelines.

In [12]:
X = X.drop(columns=['AUCGUART', 'PRIMEUNIT'])

## Type of Variables

In [13]:
num_cols = X.select_dtypes(exclude=['object','category']).columns
binary_cols = [col for col in X.columns if X[col].isin([0, 1, 0.0, 1.0, '1','0']).all()]
num_cols = [num_col for num_col in num_cols if num_col not in binary_cols]

print(f'Numeric columns: {len(num_cols)}')
print(f'Binary columns: {len(binary_cols)}')

Numeric columns: 14
Binary columns: 1


In [14]:
for col in binary_cols:
    X[col] = X[col].astype(float)

In [15]:
cat_columns = X.select_dtypes(include=['category','object']).columns
print(f'Category columns: {len(cat_columns)}')

Category columns: 15


General review of the values of all variables.

In [16]:
X[X.select_dtypes(include=['category']).columns].apply(lambda col: col.nunique()).sort_values(ascending=False)

Model                   1063
SubModel                 863
VNZIP1                   153
Trim                     134
BYRNO                     74
VNST                      37
Make                      33
Color                     16
Size                      12
WheelTypeID                4
Nationality                4
TopThreeAmericanName       4
Auction                    3
Transmission               3
WheelType                  3
dtype: int64

## Value counts of the variables with more cardinality

#### Model

In [17]:
X.Model.value_counts()[0:10]

Model
PT CRUISER              2329
IMPALA                  1990
TAURUS                  1425
CALIBER                 1375
CARAVAN GRAND FWD V6    1289
MALIBU 4C               1225
TAURUS 3.0L V6 EFI      1160
SEBRING 4C              1157
COBALT                  1106
PT CRUISER 2.4L I4 S    1104
Name: count, dtype: int64

#### SubModel

In [18]:
X.SubModel.value_counts()[0:10]

SubModel
4D SEDAN            15236
4D SEDAN LS          4718
4D SEDAN SE          3859
4D WAGON             2230
MINIVAN 3.3L         1258
4D SUV 4.2L LS       1193
4D SEDAN LT          1129
4D SEDAN SXT FFV     1094
2D COUPE             1072
4D SEDAN LX          1068
Name: count, dtype: int64

#### VNZIP1

In [19]:
X.VNZIP1.value_counts()[0:10]

VNZIP1
32824    3699
27542    3402
75236    2431
74135    2321
80022    2118
85226    2086
85040    2012
29697    1999
95673    1970
28273    1887
Name: count, dtype: int64

## Response variable distribution

In [20]:
y.value_counts()

IsBadBuy
0    64007
1     8976
Name: count, dtype: int64

In [21]:
y.value_counts(normalize=True)

IsBadBuy
0    0.877012
1    0.122988
Name: proportion, dtype: float64

## Train-Test Split

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size = 0.33, 
                                                    random_state = 42,
                                                    stratify = y)

## Pipelines (Encoding in all variables)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import balanced_accuracy_score

import scipy.stats
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

import time

In [25]:
num_cols = X_train.select_dtypes(include=['number']).columns.to_list()
cat_cols = X_train.select_dtypes(include=['category','object']).columns.to_list()

In [26]:
cat_cols_less9 = [col for col in cat_cols if X[col].nunique() <= 9]
cat_cols_more9 = [col for col in cat_cols if X[col].nunique() > 9]

In [27]:
# Define the HistGradientBoostingClassifier models
hgb_default = HistGradientBoostingClassifier(max_iter=1000,
                                             random_state=1234,
                                             early_stopping=False,
                                             scoring='balanced_accuracy',
                                             class_weight='balanced')

hgb_default_es = HistGradientBoostingClassifier(max_iter=1000,
                                                random_state=1234,
                                                early_stopping=True,
                                                validation_fraction=0.1,
                                                n_iter_no_change=5,
                                                scoring='balanced_accuracy',
                                                class_weight='balanced')

hgb_default9 = HistGradientBoostingClassifier(max_iter=1000,
                                              random_state=1234,
                                              early_stopping=False,
                                              scoring='balanced_accuracy',
                                              class_weight='balanced')

hgb_default9_es = HistGradientBoostingClassifier(max_iter=1000,
                                                 random_state=1234,
                                                 early_stopping=True,
                                                 validation_fraction=0.1,
                                                 n_iter_no_change=5,
                                                 scoring='balanced_accuracy',
                                                 class_weight='balanced')

# Define the hyperparameter search space
param_distributions = {
    'model__learning_rate': scipy.stats.uniform(0.01, 0.3),
    'model__min_samples_leaf': scipy.stats.randint(10, 50),
}

# Create a StratifiedKFold cross-validation instance
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

### One Hot Encoding + HistGradientBoosting

#### Preprocessing

In [28]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "median"))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot", cat_pipeline, cat_cols)],
    sparse_threshold=0
)

#### Default parameters

In [29]:
ohe_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                     ('model', hgb_default)])

ohe_hgb_default_es_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                        ('model', hgb_default_es)])

##### No early stopping

In [30]:
tic = time.time()

ohe_hgb_default = ohe_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
ohe_hgb_default_time_taken = toc-tic

In [31]:
# Display pipeline
print("Time taken: ", ohe_hgb_default_time_taken)
ohe_hgb_default

Time taken:  584.876042842865


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [32]:
# # Calculate balanced accuracy for the model with default parameters
y_ohe_hgb_default_pred = ohe_hgb_default.predict(X_test)
ohe_hgb_default_accuracy = balanced_accuracy_score(y_test, y_ohe_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {ohe_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("OneHotEncoding")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(ohe_hgb_default_accuracy)
time_summary.append(ohe_hgb_default_time_taken)
n_models_summary.append(1)

c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [2, 3, 4, 12, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Balanced accuracy with default parameters: 0.6306001410407254


##### With early stopping

In [33]:
tic = time.time()

ohe_hgb_default_es = ohe_hgb_default_es_pipeline.fit(X_train, y_train)

toc = time.time()
ohe_hgb_default_es_time_taken = toc-tic

In [34]:
# Display pipeline
print("Time taken: ", ohe_hgb_default_es_time_taken)
ohe_hgb_default_es

Time taken:  13.496708154678345


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                                 handle_unknown='ignore'))]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [35]:
# # Calculate balanced accuracy for the model with default parameters
y_ohe_hgb_default_es_pred = ohe_hgb_default_es.predict(X_test)
ohe_hgb_default_es_accuracy = balanced_accuracy_score(y_test, y_ohe_hgb_default_es_pred)
print(f'Balanced accuracy with default parameters: {ohe_hgb_default_es_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("OneHotEncoding")
early_stopping_summary.append("EarlyStopping")
value_summary.append(ohe_hgb_default_es_accuracy)
time_summary.append(ohe_hgb_default_es_time_taken)
n_models_summary.append(1)

c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [2, 3, 4, 12, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Balanced accuracy with default parameters: 0.64068328065164


#### Hyperparameter tunning

In [36]:
ohe_hgb_tune = RandomizedSearchCV(estimator = ohe_hgb_default_es_pipeline, 
                                  param_distributions = param_distributions, 
                                  n_iter = 20,
                                  cv = stratified_kfold,
                                  scoring = 'balanced_accuracy', 
                                  random_state = 1234,
                                  n_jobs = -1)

In [37]:
tic = time.time()

ohe_hgb_tune = ohe_hgb_tune.fit(X_train, y_train)

toc = time.time()
ohe_hgb_tune_time_taken = toc-tic

In [38]:
# Display pipeline
print("Time taken: ", ohe_hgb_tune_time_taken)
ohe_hgb_tune

Time taken:  235.94227004051208


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

In [39]:
# Get the best parameters
ohe_hgb_tune_best_params = ohe_hgb_tune.best_params_
print(f'Best parameters: {ohe_hgb_tune_best_params}')

# Predict using the model with the best parameters
y_ohe_hgb_tune_pred = ohe_hgb_tune.predict(X_test)
ohe_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_ohe_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {ohe_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("OneHotEncoding")
early_stopping_summary.append("EarlyStopping")
value_summary.append(ohe_hgb_tune_accuracy)
time_summary.append(ohe_hgb_tune_time_taken)
n_models_summary.append(ohe_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.22137439245687177, 'model__min_samples_leaf': 41}


c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [2, 3, 4, 12, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Balanced accuracy with best parameters: 0.6548800388247187


### Count Encoder + HistGradientBoosting


In [40]:
from category_encoders.count import CountEncoder

#### Preprocessing

In [41]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", CountEncoder())
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("count_encoder", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("count_encoder", cat_pipeline_more9, cat_cols_more9),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9)],
    sparse_threshold=0
)

#### Default hyper-parameters

In [42]:
count_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                       ('model', hgb_default)])
count_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                        ('model', hgb_default9)])

count_hgb_default_es_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                         ('model', hgb_default_es)])
count_hgb_default_es_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                        ('model', hgb_default9_es)])

##### Without early-stopping

All variables

In [43]:
tic = time.time()

count_hgb_default = count_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
count_hgb_default_time_taken = toc-tic

In [44]:
# Display pipeline
print("Time taken: ", count_hgb_default_time_taken)
count_hgb_default

Time taken:  16.287976264953613


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [45]:
tic = time.time()

count_hgb_default9 = count_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
count_hgb_default9_time_taken = toc-tic

In [46]:
# Display pipeline
print("Time taken: ", count_hgb_default9_time_taken)
count_hgb_default9

Time taken:  18.306907176971436


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Auction', 'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality',
                                                   'TopThreeAmericanName'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Results

In [47]:
# Calculate balanced accuracy for the model with default parameters
y_count_hgb_default_pred = count_hgb_default.predict(X_test)
count_hgb_default_accuracy = balanced_accuracy_score(y_test, y_count_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {count_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("CountEncoding")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(count_hgb_default_accuracy)
time_summary.append(count_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6141437807935214


In [48]:
# Calculate balanced accuracy for the model with default parameters
y_count_hgb_default_pred9 = count_hgb_default9.predict(X_test)
count_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_count_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {count_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CountEncoding")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(count_hgb_default_accuracy9)
time_summary.append(count_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6110653676547989


##### With early-stopping

All variables

In [49]:
tic = time.time()

count_hgb_default_es = count_hgb_default_es_pipeline.fit(X_train, y_train)

toc = time.time()
count_hgb_default_es_time_taken = toc-tic

In [50]:
# Display pipeline
print("Time taken: ", count_hgb_default_es_time_taken)
count_hgb_default_es

Time taken:  2.5400547981262207


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [51]:
tic = time.time()

count_hgb_default9_es = count_hgb_default_es_pipeline9.fit(X_train, y_train)

toc = time.time()
count_hgb_default9_es_time_taken = toc-tic

In [52]:
# Display pipeline
print("Time taken: ", count_hgb_default9_es_time_taken)
count_hgb_default9_es

Time taken:  1.967989444732666


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Auction', 'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality',
                                                   'TopThreeAmericanName'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [53]:
# Calculate balanced accuracy for the model with default parameters
y_count_hgb_default_es_pred = count_hgb_default_es.predict(X_test)
count_hgb_default_es_accuracy = balanced_accuracy_score(y_test, y_count_hgb_default_es_pred)
print(f'Balanced accuracy with default parameters: {count_hgb_default_es_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("CountEncoding")
early_stopping_summary.append("EarlyStopping")
value_summary.append(count_hgb_default_es_accuracy)
time_summary.append(count_hgb_default_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.644407736199821


In [54]:
# Calculate balanced accuracy for the model with default parameters
y_count_hgb_default_es_pred9 = count_hgb_default9_es.predict(X_test)
count_hgb_default_es_accuracy9 = balanced_accuracy_score(y_test, y_count_hgb_default_es_pred9)
print(f'Balanced accuracy with default parameters: {count_hgb_default_es_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CountEncoding")
early_stopping_summary.append("EarlyStopping")
value_summary.append(count_hgb_default_es_accuracy9)
time_summary.append(count_hgb_default9_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6380440894036195


#### Tuning hyper-parameters

In [55]:
count_hgb_tune = RandomizedSearchCV(estimator = count_hgb_default_es_pipeline, 
                                    param_distributions = param_distributions, 
                                    n_iter = 20,
                                    cv = stratified_kfold,
                                    scoring = 'balanced_accuracy', 
                                    random_state = 1234,
                                    n_jobs = -1)

count_hgb_tune9 = RandomizedSearchCV(estimator = count_hgb_default_es_pipeline9, 
                                     param_distributions = param_distributions, 
                                     n_iter = 20,
                                     cv = stratified_kfold,
                                     scoring = 'balanced_accuracy', 
                                     random_state = 1234,
                                     n_jobs = -1)

In [56]:
tic = time.time()

count_hgb_tune = count_hgb_tune.fit(X_train, y_train)

toc = time.time()
count_hgb_tune_time_taken = toc-tic

In [57]:
# Display pipeline
print("Time taken: ", count_hgb_tune_time_taken)
count_hgb_tune

Time taken:  31.884937524795532


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

In [58]:
tic = time.time()

count_hgb_tune9 = count_hgb_tune9.fit(X_train, y_train)

toc = time.time()
count_hgb_tune9_time_taken = toc-tic

In [59]:
# Display pipeline
print("Time taken: ", count_hgb_tune9_time_taken)
count_hgb_tune9

Time taken:  28.271488904953003


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

In [60]:
# Predict using the model with the best parameters
y_count_hgb_tune_pred = count_hgb_tune.predict(X_test)

# Get the best parameters
count_hgb_tune_best_params = count_hgb_tune.best_params_
print(f'Best parameters: {count_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
count_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_count_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {count_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("CountEncoding")
early_stopping_summary.append("EarlyStopping")
value_summary.append(count_hgb_tune_accuracy)
time_summary.append(count_hgb_tune_time_taken)
n_models_summary.append(count_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.2553515323349708, 'model__min_samples_leaf': 34}
Balanced accuracy with best parameters: 0.6535370160619628


In [61]:
# Predict using the model with the best parameters
y_count_hgb_tune_pred9 = count_hgb_tune9.predict(X_test)

# Get the best parameters
count_hgb_tune_best_params9 = count_hgb_tune9.best_params_
print(f'Best parameters: {count_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
count_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_count_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {count_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CountEncoding")
early_stopping_summary.append("EarlyStopping")
value_summary.append(count_hgb_tune_accuracy9)
time_summary.append(count_hgb_tune9_time_taken)
n_models_summary.append(count_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.1458189427790103, 'model__min_samples_leaf': 27}
Balanced accuracy with best parameters: 0.6385995319590925


### Ordinal Encoding + HistGradientBoosting

In [62]:
from sklearn.preprocessing import OrdinalEncoder

#### Preprocessing

In [63]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OrdinalEncoder(dtype = int,
                               handle_unknown = 'use_encoded_value',
                               unknown_value = 99999,
                               encoded_missing_value = 99999))
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("ordinal_encoder", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("ordinal_encoder", cat_pipeline_more9, cat_cols_more9),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9)],
    sparse_threshold=0
)

#### Default hyper-parameters

In [64]:
ordinal_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                         ('model', hgb_default)])

ordinal_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                          ('model', hgb_default9)])

ordinal_hgb_default_es_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                         ('model', hgb_default_es)])

ordinal_hgb_default_es_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                        ('model', hgb_default9_es)])

##### Without early-stopping

In [65]:
tic = time.time()

ordinal_hgb_default = ordinal_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_default_time_taken = toc-tic

In [66]:
# Display pipeline
print("Time taken: ", ordinal_hgb_default_time_taken)
ordinal_hgb_default

Time taken:  14.949808359146118


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=99999))]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [67]:
tic = time.time()

ordinal_hgb_default9 = ordinal_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_default9_time_taken = toc-tic

In [68]:
# Display pipeline
print("Time taken: ", ordinal_hgb_default9_time_taken)
ordinal_hgb_default9

Time taken:  17.739381551742554


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Auction', 'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality',
                                                   'TopThreeAmericanName'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [69]:
# Calculate balanced accuracy for the model with default parameters
y_ordinal_hgb_default_pred = ordinal_hgb_default.predict(X_test)
ordinal_hgb_default_accuracy = balanced_accuracy_score(y_test, y_ordinal_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {ordinal_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("OrdinalEncoder")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(ordinal_hgb_default_accuracy)
time_summary.append(ordinal_hgb_default_time_taken)
n_models_summary.append(1)


Balanced accuracy with default parameters: 0.6151217030068219


In [70]:
# Calculate balanced accuracy for the model with default parameters
y_ordinal_hgb_default_pred9 = ordinal_hgb_default9.predict(X_test)
ordinal_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_ordinal_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {ordinal_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("OrdinalEncoder")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(ordinal_hgb_default_accuracy9)
time_summary.append(ordinal_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6201260722900686


##### With early-stopping

All variables

In [71]:
tic = time.time()

ordinal_hgb_default_es = ordinal_hgb_default_es_pipeline.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_default_es_time_taken = toc-tic

In [72]:
# Display pipeline
print("Time taken: ", ordinal_hgb_default_es_time_taken)
ordinal_hgb_default_es

Time taken:  1.682939052581787


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                                  unknown_value=99999))]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [73]:
tic = time.time()

ordinal_hgb_default9_es = ordinal_hgb_default_es_pipeline9.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_default9_es_time_taken = toc-tic

In [74]:
# Display pipeline
print("Time taken: ", ordinal_hgb_default9_es_time_taken)
ordinal_hgb_default9_es

Time taken:  2.0795812606811523


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Auction', 'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality',
                                                   'TopThreeAmericanName'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Results

In [75]:
# Calculate balanced accuracy for the model with default parameters
y_ordinal_hgb_default_es_pred = ordinal_hgb_default.predict(X_test)
ordinal_hgb_default_es_accuracy = balanced_accuracy_score(y_test, y_ordinal_hgb_default_es_pred)
print(f'Balanced accuracy with default parameters: {ordinal_hgb_default_es_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("OrdinalEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(ordinal_hgb_default_es_accuracy)
time_summary.append(ordinal_hgb_default_es_time_taken)
n_models_summary.append(1)


Balanced accuracy with default parameters: 0.6151217030068219


In [76]:
# Calculate balanced accuracy for the model with default parameters
y_ordinal_hgb_default_es_pred9 = ordinal_hgb_default9_es.predict(X_test)
ordinal_hgb_default_es_accuracy9 = balanced_accuracy_score(y_test, y_ordinal_hgb_default_es_pred9)
print(f'Balanced accuracy with default parameters: {ordinal_hgb_default_es_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("OrdinalEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(ordinal_hgb_default_es_accuracy9)
time_summary.append(ordinal_hgb_default9_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6458334951615985


#### Hyper-parameter tunning

In [77]:
ordinal_hgb_tune = RandomizedSearchCV(estimator = ordinal_hgb_default_es_pipeline, 
                                      param_distributions = param_distributions, 
                                      n_iter = 20,
                                      cv = stratified_kfold,
                                      scoring = 'balanced_accuracy', 
                                      random_state = 1234,
                                      n_jobs = -1)

ordinal_hgb_tune9 = RandomizedSearchCV(estimator = ordinal_hgb_default_es_pipeline9, 
                                       param_distributions = param_distributions, 
                                       n_iter = 20,
                                       cv = stratified_kfold,
                                       scoring = 'balanced_accuracy', 
                                       random_state = 1234,
                                       n_jobs = -1)

All variables

In [78]:
tic = time.time()

ordinal_hgb_tune = ordinal_hgb_tune.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_tune_time_taken = toc-tic

In [79]:
# Display pipeline
print("Time taken: ", ordinal_hgb_tune_time_taken)
ordinal_hgb_tune

Time taken:  25.23456835746765


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

Variables >= 9

In [80]:
tic = time.time()

ordinal_hgb_tune9 = ordinal_hgb_tune9.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_tune9_time_taken = toc-tic

In [81]:
# Display pipeline
print("Time taken: ", ordinal_hgb_tune9_time_taken)
ordinal_hgb_tune9

Time taken:  21.90940761566162


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

Results

In [82]:
# Predict using the model with the best parameters
y_ordinal_hgb_tune_pred = ordinal_hgb_tune.predict(X_test)

# Get the best parameters
ordinal_hgb_tune_best_params = ordinal_hgb_tune.best_params_
print(f'Best parameters: {ordinal_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
ordinal_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_ordinal_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {ordinal_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("OrdinalEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(ordinal_hgb_tune_accuracy)
time_summary.append(ordinal_hgb_tune_time_taken)
n_models_summary.append(ordinal_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.27479235719083495, 'model__min_samples_leaf': 49}
Balanced accuracy with best parameters: 0.648497963585076


In [83]:
# Predict using the model with the best parameters
y_ordinal_hgb_tune_pred9 = ordinal_hgb_tune9.predict(X_test)

# Get the best parameters
ordinal_hgb_tune_best_params9 = ordinal_hgb_tune9.best_params_
print(f'Best parameters: {ordinal_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
ordinal_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_ordinal_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {ordinal_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("OrdinalEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(ordinal_hgb_tune_accuracy9)
time_summary.append(ordinal_hgb_tune9_time_taken)
n_models_summary.append(ordinal_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.2553515323349708, 'model__min_samples_leaf': 34}
Balanced accuracy with best parameters: 0.6534294102549669


### Native HistGradientBoosting support for categorical variables


In [84]:
from sklearn.preprocessing import OrdinalEncoder

#### Preprocessing

In [85]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OrdinalEncoder(dtype = int,
                               handle_unknown = 'use_encoded_value',
                               unknown_value = 99999,
                               encoded_missing_value = 99999,
                               max_categories = 254))
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("cat", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9),
                    ("ordinal_encoding", cat_pipeline_more9, cat_cols_more9)],
    sparse_threshold=0
)

In [86]:
category_features_for_nativesupport = [False]*len(num_cols) + [True]*len(cat_cols)

X_train_check = preprop_pipeline9.fit_transform(X_train)
category_features_for_nativesupport9 = [False]*(X_train_check.shape[1]-len(cat_cols_more9)) + [True]*len(cat_cols_more9)
print(category_features_for_nativesupport)
print(category_features_for_nativesupport9)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True]


In [88]:
hgb_default_categories_support = HistGradientBoostingClassifier(max_iter=1000,
                                                                random_state=1234,
                                                                early_stopping=False,
                                                                scoring='balanced_accuracy',
                                                                class_weight='balanced',
                                                                categorical_features=category_features_for_nativesupport)

hgb_default_categories_support9 = HistGradientBoostingClassifier(max_iter=1000,
                                                                 random_state=1234,
                                                                 early_stopping=False,
                                                                 scoring='balanced_accuracy',
                                                                 class_weight='balanced',
                                                                 categorical_features=category_features_for_nativesupport9)

hgb_default_es_categories_support = HistGradientBoostingClassifier(max_iter=1000,
                                                                  random_state=1234,
                                                                  early_stopping=True,
                                                                  validation_fraction=0.1,
                                                                  n_iter_no_change=5,
                                                                  scoring='balanced_accuracy',
                                                                  class_weight='balanced',
                                                                  categorical_features=category_features_for_nativesupport)

hgb_default_es_categories_support9 = HistGradientBoostingClassifier(max_iter=1000,
                                                                    random_state=1234,
                                                                    early_stopping=True,
                                                                    validation_fraction=0.1,
                                                                    n_iter_no_change=5,
                                                                    scoring='balanced_accuracy',
                                                                    class_weight='balanced',
                                                                    categorical_features=category_features_for_nativesupport9)

#### Default hyper-parameters

In [89]:
catsup_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                        ('model', hgb_default_categories_support)])

catsup_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                         ('model', hgb_default_categories_support9)])

catsup_hgb_default_es_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                           ('model', hgb_default_es_categories_support)])

catsup_hgb_default_es_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                            ('model', hgb_default_es_categories_support9)])

##### Without early-stopping

All variables

In [90]:
tic = time.time()

catsup_hgb_default = catsup_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
catsup_hgb_default_time_taken = toc-tic

In [91]:
# Display pipeline
print("Time taken: ", catsup_hgb_default_time_taken)
catsup_hgb_default

Time taken:  13.644458532333374


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(categorical_features=[False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True],
                                                class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [92]:
tic = time.time()

catsup_hgb_default9 = catsup_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
catsup_hgb_default9_time_taken = toc-tic

In [93]:
# Display pipeline
print("Time taken: ", catsup_hgb_default9_time_taken)
catsup_hgb_default9

Time taken:  14.310847043991089


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                ('model',
                 HistGradientBoostingClassifier(categorical_features=[False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False, ...],
                                                class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Results

In [94]:
# Calculate balanced accuracy for the model with default parameters
y_catsup_hgb_default_pred = catsup_hgb_default.predict(X_test)
catsup_hgb_default_accuracy = balanced_accuracy_score(y_test, y_catsup_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {catsup_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("HGB_NativeSupport")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(catsup_hgb_default_accuracy)
time_summary.append(catsup_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.5651044860777026


In [95]:
# Calculate balanced accuracy for the model with default parameters
y_catsup_hgb_default_pred9 = catsup_hgb_default9.predict(X_test)
catsup_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_catsup_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {catsup_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("HGB_NativeSupport")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(catsup_hgb_default_accuracy9)
time_summary.append(catsup_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.5676695559844764


##### With early-stopping

All variables

In [96]:
tic = time.time()

catsup_hgb_default_es = catsup_hgb_default_es_pipeline.fit(X_train, y_train)

toc = time.time()
catsup_hgb_default_es_time_taken = toc-tic

In [97]:
# Display pipeline
print("Time taken: ", catsup_hgb_default_es_time_taken)
catsup_hgb_default_es

Time taken:  1.9800548553466797


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                ('model',
                 HistGradientBoostingClassifier(categorical_features=[False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True],
                                                class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [98]:
tic = time.time()

catsup_hgb_default9_es = catsup_hgb_default_es_pipeline9.fit(X_train, y_train)

toc = time.time()
catsup_hgb_default9_es_time_taken = toc-tic

In [99]:
# Display pipeline
print("Time taken: ", catsup_hgb_default9_es_time_taken)
catsup_hgb_default9_es

Time taken:  1.9046945571899414


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                 HistGradientBoostingClassifier(categorical_features=[False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False, ...],
                                                class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Results

In [100]:
# Calculate balanced accuracy for the model with default parameters
y_catsup_hgb_default_es_pred = catsup_hgb_default_es.predict(X_test)
catsup_hgb_default_es_accuracy = balanced_accuracy_score(y_test, y_catsup_hgb_default_es_pred)
print(f'Balanced accuracy with default parameters: {catsup_hgb_default_es_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("HGB_NativeSupport")
early_stopping_summary.append("EarlyStopping")
value_summary.append(catsup_hgb_default_es_accuracy)
time_summary.append(catsup_hgb_default_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6321678613508488


In [101]:
# Calculate balanced accuracy for the model with default parameters
y_catsup_hgb_default_es_pred9 = catsup_hgb_default9_es.predict(X_test)
catsup_hgb_default_es_accuracy9 = balanced_accuracy_score(y_test, y_catsup_hgb_default_es_pred9)
print(f'Balanced accuracy with default parameters: {catsup_hgb_default_es_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("HGB_NativeSupport")
early_stopping_summary.append("EarlyStopping")
value_summary.append(catsup_hgb_default_es_accuracy9)
time_summary.append(catsup_hgb_default9_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6322917219080436


#### Hyper-parameter tuning

In [102]:
catsup_hgb_tune = RandomizedSearchCV(estimator = catsup_hgb_default_es_pipeline, 
                                     param_distributions = param_distributions, 
                                     n_iter = 20,
                                     cv = stratified_kfold,
                                     scoring = 'balanced_accuracy', 
                                     random_state = 1234,
                                     n_jobs = -1)

catsup_hgb_tune9 = RandomizedSearchCV(estimator = catsup_hgb_default_es_pipeline9, 
                                      param_distributions = param_distributions, 
                                      n_iter = 20,
                                      cv = stratified_kfold,
                                      scoring = 'balanced_accuracy', 
                                      random_state = 1234,
                                     n_jobs = -1)

All variables

In [103]:
tic = time.time() 

catsup_hgb_tune = catsup_hgb_tune.fit(X_train, y_train)

toc = time.time()
catsup_hgb_tune_time_taken = toc-tic

In [104]:
# Display pipeline
print("Time taken: ", catsup_hgb_tune_time_taken)
catsup_hgb_tune

Time taken:  13.48118281364441


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

Variables >= 9

In [105]:
tic = time.time() 

catsup_hgb_tune9 = catsup_hgb_tune9.fit(X_train, y_train)

toc = time.time()
catsup_hgb_tune9_time_taken = toc-tic

c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\VNG\AppData\Local\Programs\Python\Python310\lib\site-packages\sklear

In [106]:
# Display pipeline
print("Time taken: ", catsup_hgb_tune9_time_taken)
catsup_hgb_tune9

Time taken:  10.909977912902832


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

Results

In [107]:
# Predict using the model with the best parameters
y_catsup_hgb_tune_pred = catsup_hgb_tune.predict(X_test)

# Get the best parameters
catsup_hgb_tune_best_params = catsup_hgb_tune.best_params_
print(f'Best parameters: {catsup_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
catsup_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_catsup_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {catsup_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("HGB_NativeSupport")
early_stopping_summary.append("EarlyStopping")
value_summary.append(catsup_hgb_tune_accuracy)
time_summary.append(catsup_hgb_tune_time_taken)
n_models_summary.append(catsup_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.1458189427790103, 'model__min_samples_leaf': 27}
Balanced accuracy with best parameters: 0.6323751693522807


In [108]:
# Predict using the model with the best parameters
y_catsup_hgb_tune_pred9 = catsup_hgb_tune9.predict(X_test)

# Get the best parameters
catsup_hgb_tune_best_params9 = catsup_hgb_tune9.best_params_
print(f'Best parameters: {catsup_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
catsup_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_catsup_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {catsup_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("HGB_NativeSupport")
early_stopping_summary.append("EarlyStopping")
value_summary.append(catsup_hgb_tune_accuracy9)
time_summary.append(catsup_hgb_tune9_time_taken)
n_models_summary.append(catsup_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.06745583511366768, 'model__min_samples_leaf': 48}
Balanced accuracy with best parameters: 0.6233735699935457


### Target Encoder (scikit-learn)

In [109]:
from sklearn.preprocessing import TargetEncoder

#### Preprocessing

In [110]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", TargetEncoder())
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("cat", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9),
                    ("target_encoding", cat_pipeline_more9, cat_cols_more9)
                    ],
    sparse_threshold=0
)

#### Default hyper-parameters

In [111]:
target_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                       ('model', hgb_default)])

target_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                       ('model', hgb_default9)])

target_hgb_default_es_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                       ('model', hgb_default_es)])

target_hgb_default_es_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                            ('model', hgb_default9_es)])

##### Without early-stopping

All variables

In [112]:
tic = time.time()

target_hgb_default = target_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
target_hgb_default_time_taken = toc-tic

In [113]:
# Display pipeline
print("Time taken: ", target_hgb_default_time_taken)
target_hgb_default

Time taken:  9.923060894012451


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [114]:
tic = time.time()

target_hgb_default9 = target_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
target_hgb_default9_time_taken = toc-tic

In [115]:
# Display pipeline
print("Time taken: ", target_hgb_default9_time_taken)
target_hgb_default9

Time taken:  11.59939455986023


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                   'TopThreeAmericanName']),
                                                 ('target_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['Make', 'Model', 'Trim',
                                                   'SubModel', 'Color', 'Size',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=False,
                                                max_iter=1000,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Results

In [116]:
# Calculate balanced accuracy for the model with default parameters
y_target_hgb_default_pred = target_hgb_default.predict(X_test)
target_hgb_default_accuracy = balanced_accuracy_score(y_test, y_target_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {target_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("TargetEncoder")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(target_hgb_default_accuracy)
time_summary.append(target_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6087396277671795


In [117]:
# Calculate balanced accuracy for the model with default parameters
y_target_hgb_default_pred9 = target_hgb_default9.predict(X_test)
target_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_target_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {target_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("TargetEncoder")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(target_hgb_default_accuracy9)
time_summary.append(target_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6125934500293337


##### With early-stopping

All variables

In [118]:
tic = time.time()

target_hgb_default_es = target_hgb_default_es_pipeline.fit(X_train, y_train)

toc = time.time()
target_hgb_default_es_time_taken = toc-tic

In [119]:
# Display pipeline
print("Time taken: ", target_hgb_default_es_time_taken)
target_hgb_default_es

Time taken:  0.8524420261383057


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   TargetEncoder())]),
                                                  ['Auction', 'Make', 'Model',
                                                   'Trim', 'SubModel', 'Color',
                                                   'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality', 'Size',
                                                   'TopThreeAmericanName',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Variables >= 9

In [120]:
tic = time.time()

target_hgb_default9_es = target_hgb_default_es_pipeline9.fit(X_train, y_train)

toc = time.time()
target_hgb_default9_es_time_taken = toc-tic

In [121]:
# Display pipeline
print("Time taken: ", target_hgb_default9_es_time_taken)
target_hgb_default9_es

Time taken:  2.913059949874878


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                 ('target_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['Make', 'Model', 'Trim',
                                                   'SubModel', 'Color', 'Size',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

Results

In [122]:
# Calculate balanced accuracy for the model with default parameters
y_target_hgb_default_es_pred = target_hgb_default_es.predict(X_test)
target_hgb_default_es_accuracy = balanced_accuracy_score(y_test, y_target_hgb_default_es_pred)
print(f'Balanced accuracy with default parameters: {target_hgb_default_es_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("TargetEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(target_hgb_default_es_accuracy)
time_summary.append(target_hgb_default_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6405878459284952


In [123]:
# Calculate balanced accuracy for the model with default parameters
y_target_hgb_default_es_pred9 = target_hgb_default9_es.predict(X_test)
target_hgb_default_es_accuracy9 = balanced_accuracy_score(y_test, y_target_hgb_default_es_pred9)
print(f'Balanced accuracy with default parameters: {target_hgb_default_es_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("TargetEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(target_hgb_default_es_accuracy9)
time_summary.append(target_hgb_default9_es_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6519998313469773


#### Hyper-parameter tuning

In [124]:
target_hgb_tune = RandomizedSearchCV(estimator = target_hgb_default_es_pipeline, 
                                     param_distributions = param_distributions, 
                                     n_iter = 20,
                                     cv = stratified_kfold,
                                     scoring = 'balanced_accuracy', 
                                     random_state = 1234,
                                     n_jobs = -1)

target_hgb_tune9 = RandomizedSearchCV(estimator = target_hgb_default_es_pipeline9, 
                                      param_distributions = param_distributions, 
                                      n_iter = 20,
                                      cv = stratified_kfold,
                                      scoring = 'balanced_accuracy', 
                                      random_state = 1234,
                                      n_jobs = -1)

All variables

In [125]:
tic = time.time() 

target_hgb_tune = target_hgb_tune.fit(X_train, y_train)

toc = time.time()
target_hgb_tune_time_taken = toc-tic

In [126]:
# Display pipeline
print("Time taken: ", target_hgb_tune_time_taken)
target_hgb_tune

Time taken:  12.256631851196289


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

Variables >=9

In [127]:
tic = time.time() 

target_hgb_tune9 = target_hgb_tune9.fit(X_train, y_train)

toc = time.time()
target_hgb_tune9_time_taken = toc-tic

In [128]:
# Display pipeline
print("Time taken: ", target_hgb_tune9_time_taken)
target_hgb_tune9

Time taken:  13.691378593444824


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                                                                             random_state=1234,
                                                                             scoring='balanced_accuracy'))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022175E5F070>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022175E5FA60>},
                   random_state=1234, scoring='balanced_accuracy')

Results

In [129]:
# Predict using the model with the best parameters
y_target_hgb_tune_pred = target_hgb_tune.predict(X_test)

# Get the best parameters
target_hgb_tune_best_params = target_hgb_tune.best_params_
print(f'Best parameters: {target_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
target_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_target_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {target_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("TargetEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(target_hgb_tune_accuracy)
time_summary.append(target_hgb_tune_time_taken)
n_models_summary.append(target_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.2506442926240477, 'model__min_samples_leaf': 23}
Balanced accuracy with best parameters: 0.6518943992332233


In [130]:
# Predict using the model with the best parameters
y_target_hgb_tune_pred9 = target_hgb_tune9.predict(X_test)

# Get the best parameters
target_hgb_tune_best_params9 = target_hgb_tune9.best_params_
print(f'Best parameters: {target_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
target_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_target_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {target_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("TargetEncoder")
early_stopping_summary.append("EarlyStopping")
value_summary.append(target_hgb_tune_accuracy9)
time_summary.append(target_hgb_tune9_time_taken)
n_models_summary.append(target_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.2150388805516409, 'model__min_samples_leaf': 19}
Balanced accuracy with best parameters: 0.6538024144169821


### CatBoost

In [131]:
from catboost import CatBoostClassifier

#### Preprocessing

In [132]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("cat_less9", cat_pipeline, cat_cols_less9),
                    ("cat_more9", cat_pipeline, cat_cols_more9)],
    sparse_threshold=0
)

In [133]:
category_features_for_catboostsupport  = [index for index in range(len(num_cols), len(num_cols) + len(cat_cols))]
category_features_for_catboostsupport9 = [index for index in range(len(num_cols) + len(cat_cols_less9), len(num_cols) + len(cat_cols))]
print(category_features_for_catboostsupport)
print(category_features_for_catboostsupport9)

[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[21, 22, 23, 24, 25, 26, 27, 28, 29]


Catboost allows to give a maximum value of unique categories for which a variable is encoded or not by One-Hot-Encoder.

In [134]:
# Create catboost models
catboost_default_raw = CatBoostClassifier(iterations=1000,
                                          eval_metric = 'BalancedAccuracy',
                                          loss_function = 'Logloss',
                                          auto_class_weights = 'Balanced',
                                          od_type='Iter',
                                          one_hot_max_size = 0,
                                          random_seed = 1234,
                                          min_data_in_leaf = 20,
                                          cat_features=category_features_for_catboostsupport,
                                          verbose = False)

catboost_default9_raw = CatBoostClassifier(iterations=1000,
                                           eval_metric = 'BalancedAccuracy',
                                           loss_function = 'Logloss',
                                           auto_class_weights = 'Balanced',
                                           od_type='Iter',
                                           one_hot_max_size = 9,
                                           random_seed = 1234,
                                           min_data_in_leaf = 20,
                                           cat_features=category_features_for_catboostsupport9,
                                           verbose = False)

# Default CatBoostClassifier Pipeline
catboost_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                        ('model', catboost_default_raw)])

catboost_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline),
                                         ('model', catboost_default9_raw)])

# Define the hyperparameter search space
catboost_param_distributions = {
    'model__iterations': scipy.stats.randint(10, 1000),
    'model__learning_rate': scipy.stats.uniform(0.01, 0.3),
    'model__min_data_in_leaf': scipy.stats.randint(10, 50),
}

# Create a StratifiedKFold cross-validation instance
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

catboost_tune_raw = RandomizedSearchCV(estimator = catboost_default_pipeline, 
                                   param_distributions = catboost_param_distributions, 
                                   n_iter = 10,
                                   cv = stratified_kfold,
                                   scoring = 'balanced_accuracy', 
                                   random_state = 1234,
                                   n_jobs = -1)


catboost_tune9_raw = RandomizedSearchCV(estimator = catboost_default_pipeline, 
                                        param_distributions = catboost_param_distributions, 
                                        n_iter = 10,
                                        cv = stratified_kfold,
                                        scoring = 'balanced_accuracy', 
                                        random_state = 1234,
                                        n_jobs = -1)

#### Default hyper-parameters

All variables

In [135]:
tic = time.time()

catboost_default = catboost_default_pipeline.fit(X_train, y_train)

toc = time.time()
catboost_default_time_taken = toc-tic

In [136]:
# Display pipeline
print("Time taken: ", catboost_default_time_taken)
catboost_default

Time taken:  68.57320094108582


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['Auction', 'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality',
                                                   'TopThreeAmericanName']),
                                                 ('cat_more9',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['Make', 'Model', 'Trim',
                                                   'SubModel', 'Color', 'Size',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x00000221780E3FD0>)])

Variables >= 9

In [137]:
tic = time.time()

catboost_default9 = catboost_default_pipeline.fit(X_train, y_train)

toc = time.time()
catboost_default_time_taken9 = toc-tic

In [138]:
# Display pipeline
print("Time taken: ", catboost_default_time_taken9)
catboost_default9

Time taken:  72.59842348098755


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['PurchDate', 'VehYear',
                                                   'VehicleAge', 'VehOdo',
                                                   'MMRAcquisitionAuctionAveragePrice',
                                                   'MMRAcquisitionAuctionCleanPrice',
                                                   'MMRAcquisitionRetailAveragePrice',
                                                   'MMRAcquisitonRetailCleanPrice',
                                                   'MMRCurrentAuc...
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['Auction', 'Transmission',
                                                   'WheelTypeID', 'WheelType',
                                                   'Nationality',
                                                   'TopThreeAmericanName']),
                                                 ('cat_more9',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['Make', 'Model', 'Trim',
                                                   'SubModel', 'Color', 'Size',
                                                   'BYRNO', 'VNZIP1',
                                                   'VNST'])])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x00000221780E3FD0>)])

Results

In [139]:
# Calculate balanced accuracy for the model with default parameters
y_catboost_default_pred = catboost_default.predict(X_test)
catboost_default_accuracy = balanced_accuracy_score(y_test, y_catboost_default_pred)
print(f'Balanced accuracy with default parameters: {catboost_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("CatboostNativeSupport")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(catboost_default_accuracy)
time_summary.append(catboost_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6701166295108969


In [140]:
# Calculate balanced accuracy for the model with default parameters
y_catboost_default_pred9 = catboost_default9.predict(X_test)
catboost_default_accuracy9 = balanced_accuracy_score(y_test, y_catboost_default_pred9)
print(f'Balanced accuracy with default parameters: {catboost_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CatboostNativeSupport")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(catboost_default_accuracy9)
time_summary.append(catboost_default_time_taken9)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.6701166295108969


#### Hyper-parameter tuning

All variables

In [141]:
tic = time.time()

catboost_tune = catboost_tune_raw.fit(X_train, y_train)

toc = time.time()
catboost_tune_time_taken = toc-tic

In [142]:
# Display pipeline
print("Time taken: ", catboost_tune_time_taken)
catboost_tune

Time taken:  716.6529490947723


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                   param_distributions={'model__iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000221780E3E80>,
                                        'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000221780E1D20>,
                                        'model__min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000221780E24A0>},
                   random_state=1234, scoring='balanced_accuracy')

Variables >= 9

In [143]:
tic = time.time()

catboost_tune9 = catboost_tune9_raw.fit(X_train, y_train)

toc = time.time()
catboost_tune_time_taken9 = toc-tic

In [144]:
# Display pipeline
print("Time taken: ", catboost_tune_time_taken9)
catboost_tune9

Time taken:  738.712450504303


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['PurchDate',
                                                                                'VehYear',
                                                                                'VehicleAge',
                                                                                'VehOdo',
                                                                                'MMRAcquisitionAuctionAveragePrice',
                                                                                'MMRAcquisitionAuctionCl...
                   param_distributions={'model__iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000221780E3E80>,
                                        'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000221780E1D20>,
                                        'model__min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000221780E24A0>},
                   random_state=1234, scoring='balanced_accuracy')

Results

In [145]:
# Predict using the model with the best parameters
y_catboost_tune_pred = catboost_tune.predict(X_test)

# Get the best parameters
catboost_tune_best_params = catboost_tune.best_params_
print(f'Best parameters: {catboost_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
catboost_tune_accuracy = balanced_accuracy_score(y_test, y_catboost_tune_pred)
print(f'Balanced accuracy with best parameters: {catboost_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("CatboostNativeSupport")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(catboost_tune_accuracy)
time_summary.append(catboost_tune_time_taken)
n_models_summary.append(catboost_tune.n_iter)

Best parameters: {'model__iterations': 346, 'model__learning_rate': 0.12064720180059235, 'model__min_data_in_leaf': 29}
Balanced accuracy with best parameters: 0.6651139464382166


In [146]:
# Predict using the model with the best parameters
y_catboost_tune_pred9 = catboost_tune9.predict(X_test)

# Get the best parameters
catboost_tune_best_params9 = catboost_tune9.best_params_
print(f'Best parameters: {catboost_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
catboost_tune_accuracy9 = balanced_accuracy_score(y_test, y_catboost_tune_pred9)
print(f'Balanced accuracy with best parameters: {catboost_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CatboostNativeSupport")
early_stopping_summary.append("NoEarlyStopping")
value_summary.append(catboost_tune_accuracy9)
time_summary.append(catboost_tune_time_taken9)
n_models_summary.append(catboost_tune9.n_iter)

Best parameters: {'model__iterations': 346, 'model__learning_rate': 0.12064720180059235, 'model__min_data_in_leaf': 29}
Balanced accuracy with best parameters: 0.6651139464382166


### Results Summary

In [147]:
results_summary = pd.DataFrame({"Dataset":"kick",
                                "Variables":card_9_summary,
                                "Default/Tune":default_summary,
                                "Encoder":encoder_summary,
                                "EarlyStopping":early_stopping_summary,
                                "Metric":"BalancedAccuracy",
                                "Value":value_summary,
                                "Time":time_summary,
                                "Iterations":n_models_summary})
results_summary["mean_Time"] = (results_summary["Time"] / results_summary["Iterations"])
results_summary

,Dataset,Variables,Default/Tune,Encoder,EarlyStopping,Metric,Value,Time,Iterations,mean_Time
0,kick,AllVariables,Default,OneHotEncoding,NoEarlyStopping,BalancedAccuracy,0.630600,584.876043,1,584.876043
1,kick,AllVariables,Default,OneHotEncoding,EarlyStopping,BalancedAccuracy,0.640683,13.496708,1,13.496708
2,kick,AllVariables,Tune,OneHotEncoding,EarlyStopping,BalancedAccuracy,0.654880,235.942270,20,11.797114
3,kick,AllVariables,Default,CountEncoding,NoEarlyStopping,BalancedAccuracy,0.614144,16.287976,1,16.287976
4,kick,OnlyVariablesWithCard>9,Default,CountEncoding,NoEarlyStopping,BalancedAccuracy,0.611065,18.306907,1,18.306907
5,kick,AllVariables,Default,CountEncoding,EarlyStopping,BalancedAccuracy,0.644408,2.540055,1,2.540055
6,kick,OnlyVariablesWithCard>9,Default,CountEncoding,EarlyStopping,BalancedAccuracy,0.638044,1.967989,1,1.967989
7,kick,AllVariables,Tune,CountEncoding,EarlyStopping,BalancedAccuracy,0.653537,31.884938,20,1.594247
8,kick,OnlyVariablesWithCard>9,Tune,CountEncoding,EarlyStopping,BalancedAccuracy,0.638600,28.271489,20,1.413574
9,kick,AllVariables,Default,OrdinalEncoder,NoEarlyStopping,BalancedAccuracy,0.615122,14.949808,1,14.949808


In [ ]:
results_summary.to_excel("kick_results.xlsx")